In [56]:
import pandas as pd
import numpy as np
import json
import requests
from typing import Dict, List, Union
from collections import defaultdict

In [57]:
# Class GameDealFetcher để lấy dữ liệu từ API
class GameDealFetcher:
    def __init__(
        self,
        api_key: str,
        game_id: str,
        shops_file: str,
        country: str = "US",
        shops: str = "61,35,16,6,20,24,37",
        since: str = "2023-11-06T00:00:00Z",
    ):
        """
        Khởi tạo đối tượng GameDealFetcher với các tham số cấu hình để lấy dữ liệu từ API.

        :param api_key: Khóa API để truy cập vào dịch vụ.
        :param game_id: ID của trò chơi cần lấy dữ liệu.
        :param shops_file: Đường dẫn tới tệp JSON chứa thông tin các cửa hàng.
        :param country: Quốc gia mà người dùng muốn lấy dữ liệu, mặc định là "US".
        :param shops: Danh sách các cửa hàng mà người dùng muốn lấy thông tin, mặc định là "61,35,16,6,20,24,37".
        :param since: Thời gian bắt đầu để lấy dữ liệu, mặc định là "2023-11-06T00:00:00Z".
        """
        self.api_key = api_key
        self.base_url = "https://api.isthereanydeal.com/games/history/v2"
        self.params = {
            "key": self.api_key,
            "id": game_id,
            "country": country,
            "shops": shops,
            "since": since,
        }
        self.shops_data = self.load_shops_data(shops_file)

    def load_shops_data(self, shops_file: str) -> Dict:
        """
        Tải dữ liệu cửa hàng từ tệp JSON.

        :param shops_file: Đường dẫn tới tệp JSON chứa thông tin các cửa hàng.
        :return: Dữ liệu cửa hàng dưới dạng từ điển.
        """
        try:
            with open(shops_file, "r", encoding="utf-8") as file:
                return json.load(file)
        except Exception as e:
            raise Exception(f"Lỗi khi đọc tệp JSON cửa hàng: {str(e)}")

    def fetch_deal_history(self) -> Union[List, Dict]:
        """
        Lấy lịch sử giảm giá từ API.

        :return: Dữ liệu lịch sử giảm giá (dạng danh sách hoặc từ điển).
        """
        response = requests.get(self.base_url, params=self.params)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Lỗi: {response.status_code} - {response.text}")

    def format_data(self, data: Union[List, Dict, str]) -> List[Dict]:
        """
        Định dạng dữ liệu từ API thành cấu trúc có tổ chức hơn để dễ xử lý.

        :param data: Dữ liệu từ API trả về (dạng danh sách, từ điển, hoặc chuỗi JSON).
        :return: Dữ liệu đã được định dạng dưới dạng danh sách các từ điển.
        """
        if isinstance(data, dict) and "data" in data:
            data = data["data"]

        formatted_data = []

        for item in data:
            shop_id = str(item["shop"]["id"])
            shop_name = self.shops_data.get(shop_id, "Cửa hàng không xác định")
            timestamp = item["timestamp"]
            deal_info = {
                "timestamp": timestamp,
                "price": item["deal"]["price"]["amount"],
                "regular_price": item["deal"]["regular"]["amount"],
                "cut": item["deal"]["cut"],
            }

            shop_entry = next(
                (s for s in formatted_data if s["shop_id"] == shop_id), None
            )

            if not shop_entry:
                shop_entry = {"shop_id": shop_id, "shop_title": shop_name, "deals": []}
                formatted_data.append(shop_entry)

            shop_entry["deals"].append(deal_info)

        return formatted_data
    
class PriceComparisonGroupedBarChart:
    def __init__(self, shops_data: List[Dict]):
        """
        Khởi tạo đối tượng PriceComparisonGroupedBarChart với dữ liệu các cửa hàng.

        :param shops_data: Dữ liệu các cửa hàng đã được định dạng.
        """
        self.shops_data = shops_data

    def process_raw_data(self) -> List[Dict]:
        """
        Xử lý dữ liệu thô để tạo dữ liệu cho biểu đồ cột nhóm, tính giá trị lớn nhất, nhỏ nhất và trung bình.
        """
        # Khởi tạo danh sách chứa tất cả các tháng có trong dữ liệu
        all_dates = set()
        for shop in self.shops_data:
            for deal in shop["deals"]:
                date = deal["timestamp"].split("T")[0][:7]
                all_dates.add(date)

        all_dates = (
            pd.date_range(start=min(all_dates), end=max(all_dates), freq="MS")
            .strftime("%Y-%m")
            .tolist()
        )

        time_point_data = defaultdict(dict)
        latest_prices = {}  # Lưu giá mới nhất của từng shop

        for date in all_dates:
            for shop in self.shops_data:
                shop_title = shop["shop_title"]

                # Lọc những deal có ngày nhỏ hơn hoặc bằng tháng hiện tại
                relevant_deals = [
                    deal
                    for deal in shop["deals"]
                    if deal["timestamp"].split("T")[0][:7] <= date
                ]

                if relevant_deals:
                    # Lấy giá trị lớn nhất và nhỏ nhất của shop trong tháng đó
                    max_deal = max(relevant_deals, key=lambda x: x["price"])
                    min_deal = min(relevant_deals, key=lambda x: x["price"])
                    avg_price = (max_deal["price"] + min_deal["price"]) / 2
                    time_point_data[date][f"{shop_title}_max"] = max_deal["price"]
                    time_point_data[date][f"{shop_title}_min"] = min_deal["price"]
                    time_point_data[date][f"{shop_title}_avg"] = avg_price
                    latest_prices[shop_title] = {"max": max_deal["price"], "min": min_deal["price"], "avg": avg_price}
                else:
                    # Nếu không có deal, lấy giá trị mới nhất đã biết
                    if shop_title in latest_prices:
                        time_point_data[date][f"{shop_title}_max"] = latest_prices[shop_title]["max"]
                        time_point_data[date][f"{shop_title}_min"] = latest_prices[shop_title]["min"]
                        time_point_data[date][f"{shop_title}_avg"] = latest_prices[shop_title]["avg"]
                    else:
                        # Nếu chưa từng có deal, để giá là 0
                        time_point_data[date][f"{shop_title}_max"] = 0
                        time_point_data[date][f"{shop_title}_min"] = 0
                        time_point_data[date][f"{shop_title}_avg"] = 0

        formatted_data = []
        for date, prices in time_point_data.items():
            entry = {"time_point": date}
            entry.update(prices)
            formatted_data.append(entry)

        # Tạo DataFrame từ formatted_data
        df = pd.DataFrame(formatted_data)
        return df


In [58]:
# Use GameDealFetcher and PriceComparisonGroupedBarChart to generate the chart
def main():
    # Necessary Information
    api_key = "07b0e806aacf15f38b230a850b424b2542dd71af"
    game_id = "018d937f-590c-728b-ac35-38bcff85f086"
    shops_file = "DiscountFrequencyAnalysis/shops.json"

    fetcher = GameDealFetcher(api_key, game_id, shops_file)
    raw_data = fetcher.fetch_deal_history()

    formatted_data = fetcher.format_data(raw_data)
    
    process_data = PriceComparisonGroupedBarChart(formatted_data)  
    df = process_data.process_raw_data()  # Không cần tạo lại DataFrame ở đây

    # Hiển thị DataFrame trực tiếp trong Jupyter Notebook
    display(df)  # Sử dụng `display()` thay vì `print(df)` để hiển thị đẹp hơn

if __name__ == "__main__":
    main()

,time_point,Steam_max,Steam_min,Steam_avg,GameBillet_max,GameBillet_min,GameBillet_avg,Fanatical_max,Fanatical_min,Fanatical_avg,Humble Store_max,Humble Store_min,Humble Store_avg,Epic Games Store_max,Epic Games Store_min,Epic Games Store_avg
0,2023-11,52.68,52.68,52.680,52.67,52.67,52.67,59.99,59.99,59.99,59.99,59.99,59.99,59.99,59.99,59.99
1,2023-12,52.68,31.79,42.235,59.99,32.13,46.06,59.99,35.99,47.99,59.99,35.99,47.99,59.99,31.67,45.83
2,2024-01,59.99,31.79,45.890,59.99,32.13,46.06,59.99,35.99,47.99,59.99,35.99,47.99,59.99,31.67,45.83
3,2024-02,59.99,31.79,45.890,59.99,32.13,46.06,59.99,35.99,47.99,59.99,35.99,47.99,59.99,31.67,45.83
4,2024-03,59.99,31.79,45.890,59.99,32.13,46.06,59.99,35.99,47.99,59.99,35.99,47.99,59.99,31.67,45.83
5,2024-04,59.99,31.79,45.890,59.99,32.13,46.06,59.99,35.99,47.99,59.99,35.99,47.99,59.99,31.67,45.83
6,2024-05,59.99,31.79,45.890,59.99,32.13,46.06,59.99,35.99,47.99,59.99,35.99,47.99,59.99,31.67,45.83
7,2024-06,59.99,31.79,45.890,59.99,32.13,46.06,59.99,35.99,47.99,59.99,35.99,47.99,59.99,31.67,45.83
8,2024-07,59.99,31.79,45.890,59.99,32.13,46.06,59.99,35.99,47.99,59.99,35.99,47.99,59.99,31.67,45.83
9,2024-08,59.99,31.79,45.890,59.99,32.13,46.06,59.99,35.99,47.99,59.99,35.99,47.99,59.99,31.67,45.83
